# Altairで「マンガ雑誌のデータ」の図を作り直す

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_4/using-altair-to-recreate-manga-plots.ipynb)

今回は、文化庁の[メディア芸術データベース・ラボ（MADB Lab）](https://mediag.bunka.go.jp/madb_lab/)で公開されている四大少年誌（週刊少年サンデー、週刊少年ジャンプ、週刊少年チャンピオン、週刊少年マガジン）のデータを使って、量的データと質的データの可視化を練習します。

まず、「四大少年誌それぞれの掲載作品のジャンルと著者にはどのような特徴があるのか？」という大きな問いを立て、可視化手法を学びながらデータを見て、具体的な問いを決めていきましょう。

[マンガと学ぶデータビジュアライゼーション](https://kakeami.github.io/viz-madb/index.html)の内容を参考にしています。

[Plotlyで作られた図](https://github.com/shinchu/dataviz-notebooks/blob/main/week_4/intro-to-visualizing-quantitative-and-qualitative-data.ipynb)の中から、好きな図を3つ選んで、Altairで再現しましょう。

再現する時には、変数の性質（名義、順序、間隔、比例）に注意して、どのタイプの図がどの変数をうまく扱えるかを考えてみましょう。

複雑な図を再現したい時は、[Altair Example Gallery](https://altair-viz.github.io/gallery/index.html)を参考にしてみてください。

## ライブラリの読み込み

In [ ]:
# ライブラリのインストール、初回のみ実行する。
!pip install altair

In [ ]:
import pandas as pd
import numpy as np

import altair as alt

In [ ]:
import itertools
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 5000行の制限をはずす
alt.data_transformers.disable_max_rows()

## 準備関数

In [ ]:
# weekdayを曜日に変換
WD2STR = {
    0: 'Mon.',
    1: 'Tue.',
    2: 'Wed.',
    3: 'Thu.',
    4: 'Fri.',
    5: 'Sat.',
    6: 'Sun.',}

In [ ]:
def add_years_to_df(df, unit_years=10):
    """unit_years単位で区切ったyears列を追加"""
    df_new = df.copy()
    df_new['years'] = \
        pd.to_datetime(df['datePublished']).dt.year \
        // unit_years * unit_years
    df_new['years'] = df_new['years'].astype(str)
    return df_new

In [ ]:
def add_weekday_to_df(df):
    """曜日情報をdfに追加"""
    df_new = df.copy()
    df_new['weekday'] = \
        pd.to_datetime(df_new['datePublished']).dt.weekday
    df_new['weekday_str'] = df_new['weekday'].apply(
        lambda x: WD2STR[x])
    return df_new

In [ ]:
def add_mcid_to_df(df):
    """mcnameのindexをdfに追加"""
    df_new = df.copy()
    mcname2mcid = {
        x: i for i, x in enumerate(df['mcname'].unique())}
    df_new['mcid'] = df_new['mcname'].apply(
        lambda x: mcname2mcid[x])
    return df_new

In [ ]:
def resample_df_by_cname_and_years(df):
    """cnameとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    cnames = df['cname'].unique()
    for cname, years in itertools.product(cnames, yearss):
        df_tmp = df_new[
            (df_new['cname'] == cname)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.DataFrame(
                {'cname': cname,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = pd.concat(
                [df_new, s], ignore_index=True)
    return df_new

In [ ]:
def resample_df_by_creator_and_years(df):
    """creatorとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    creators = df['creator'].unique()
    for creator, years in itertools.product(creators, yearss):
        df_tmp = df_new[
            (df_new['creator'] == creator)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'creator': creator,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = pd.concat(
                [df_new, s], ignore_index=True)
    return df_new

## データの用意

四大少年誌の`1970-07-27`から`2017-07-06`までの全ての掲載作品のデータを使います。

すでに前処理がされているデータがあるので、そちらを使います。

In [ ]:
file = "https://raw.githubusercontent.com/shinchu/dataviz-notebooks/main/data/week_4/episodes.csv"

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.shape

各週の掲載作品を一行ずつ格納しているため、合計で約18万行程度になります。

In [ ]:
df.columns

- `mcname`: 雑誌名（**M**gazine **C**ollection **NAME**）
- `miid`：雑誌巻号ID（**M**agazine **I**tem **ID**）
- `miname`: 雑誌巻号名（**M**agazine **I**tem **NAME**）
- `cid`: マンガ作品ID（**C**omic **ID**）
- `cname`: マンガ作品名（**C**omic **NAME**）
- `epname`: 各話タイトル（**EP**isode **NAME**）
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の出版社
- `editor`: 雑誌の編集者（編集長）
- `pages`: 各話のページ数（`pageEnd` - `pageStart` + 1）
- `pageEndMax`: 雑誌に掲載されているマンガ作品のうち，`pageEnd`の最大値
- `pageStartPosition`: 各話の`pageStart`の相対的な位置（`pageStart` / `pageEndMax`）

In [ ]:
df.head()

In [ ]:
df.describe()

欠損値を確認してみます。

特に`epname`と`publisher`の欠測が多いことがわかります．

In [ ]:
df.isna().sum().reset_index()

## （例）Altairで再現1：作品別の掲載週数（上位20作品）

In [ ]:
# 作品ごとの週数を数える
df_plot = df.value_counts("cname").reset_index(name="weeks").head(20)

In [ ]:
alt.Chart(df_plot).mark_bar().encode(
    alt.X("cname:N", sort="-y", title="作品名"),
    alt.Y("weeks:Q", title="掲載週数"),
    alt.Tooltip(["weeks"])
).properties(
    title="作品ごとの掲載週数"
).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_title(
    fontSize=16
)

## （例）Altairで再現2：長期連載作品の掲載位置の分布

In [ ]:
df_tmp = \
    df.groupby('cname')['pageStartPosition']\
    .agg(['count', 'mean']).reset_index()
df_tmp = \
    df_tmp.sort_values('count', ascending=False, ignore_index=True)\
    .head(10)
cname2position = df_tmp.groupby('cname')['mean'].first().to_dict()

In [ ]:
df_plot = df[df['cname'].isin(list(cname2position.keys()))]\
    .reset_index(drop=True)
df_plot['position'] = df_plot['cname'].apply(
    lambda x: cname2position[x])
df_plot = df_plot.sort_values('position', ignore_index=True)

In [ ]:
alt.Chart(df_plot).transform_density(
    'pageStartPosition',
    as_=['pageStartPosition', 'density'],
    extent=[0, 1],
    groupby=['cname']
).mark_area(orient='horizontal').encode(
    alt.Y('pageStartPosition:Q'),
    alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0], grid=False, ticks=True)
    ),
    alt.Column(
        'cname:N'
    ),
    alt.Tooltip(['pageStartPosition:Q'])
).properties(
    width=100
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

---

## Altairで再現1：[図のタイトル]

In [ ]:
# your code goes here

## Altairで再現2：[図のタイトル]

In [ ]:
# your code goes here

## Altairで再現3：[図のタイトル]

最後は、自分のデザインガイドを作ってみましょう。

1. まずは、以下のそれぞれの要素について、自分の方針を書いてください。

```
* 図の構成要素
    * 全体のサイズ
    * 図番号
    * タイトル
    * サブタイトル
    * 軸タイトル
    * 軸ラベル
    * 軸と目盛り
    * グリッド線
    * データソース・注記
    * ロゴ
    * 凡例
    * ラベル
* 色
* 保存形式
* アクセシビリティ
```

（例）
```
* 図の構成要素
    * 全体のサイズ：650〜900px
    * 図番号：つけない
    * タイトル：上部につける
    * サブタイトル：タイトル下部につける
    * 軸タイトル：つける
    * 軸ラベル：つける
    * 軸と目盛り：つける
    * グリッド線：横方向のみつける
    * データソース・注記：下部につける
    * ロゴ：つけない
    * 凡例：必要に応じてつける
    * ラベル：必要に応じてつける
* 色
    * データに基づいてカラースケールを決める
    * CVDセーフな配色を使う
* 保存形式：PDF
* アクセシビリティ：
    * 文字を大きくする
    * 行間を開ける
    * CVDセーフな配色を使う
```

2. 次に、このデザインガイドにしたがって図を作ってみましょう。分からないところがあったら、[Altairのドキュメント](https://altair-viz.github.io/index.html)を調べてみてください。

In [ ]:
# your code goes here